In [378]:
from fractions import Fraction
from typing import List, Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [379]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present (though your files use decimals).
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)

In [380]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

# Phương pháp lũy thừa & xuống thang - tìm giá trị riêng và vecto riêng

## Thuật toán



**1. Đầu vào (Input)**
- Ma trận vuông $A\in\mathbb{R}^{n\times n}$, có $n$ giá trị riêng thực và hệ vector riêng độc lập tuyến tính.
- Vector khởi tạo $x^{(0)}\in\mathbb{R}^n$ (ví dụ $x^{(0)}=(1,1,\dots,1)^T$).
- Ngưỡng dừng $\varepsilon>0$.
- Số lần lặp tối đa `max_iter`.
- Chuẩn $p\in\{1,2,\infty\}$ để chuẩn hóa vector riêng.

**2. Đầu ra (Output)**
- Dãy giá trị riêng $\{\lambda_1,\lambda_2,\dots,\lambda_n\}$, sắp xếp theo thứ tự $|\lambda_1|\ge|\lambda_2|\ge\cdots\ge|\lambda_n|$.
- Các vector riêng tương ứng $\{v_1,v_2,\dots,v_n\}$, với $\|v_k\|_p=1$.

---

**3. Thuật toán tổng quát**

1. **Khởi tạo**  
   $M \leftarrow A$,  $y \leftarrow x^{(0)}$.  
   Tạo danh sách rỗng `eigs=[]`, `vecs=[]`.

2. **Lặp lấy từng giá trị riêng**  
   Với $k=1,2,\dots,n$:

   1. **Chọn loại Power Method**  
      - Theo hiện tượng “dao động dấu” của loạt $\{A^m y\}$, hoặc theo chuẩn mũ xác định, phân biệt hai trường hợp:
        - **Case 1:** Giá trị riêng trội đơn ( $\lvert\lambda_1\rvert>\lvert\lambda_2\rvert$ ).  
        - **Case 2:** Hai giá trị riêng trội bằng nhau về độ lớn và ngược dấu ( $\lvert\lambda_1\rvert=\lvert\lambda_2\rvert>\lvert\lambda_3\rvert$, $\lambda_2=-\lambda_1$ ).

   2. **Tính giá trị và vector riêng trội**  
      - **Nếu Case 1**, tính dãy:  
        $
          y^{(m)} = A\,y^{(m-1)},\quad
          \lambda^{(m)} = \left\|y^{(m)}\right\|_p,\quad
          v^{(m)} = \frac{y^{(m)}}{\lambda^{(m)}}.
        $
        Lặp đến khi $\left|\lambda^{(m)}-\lambda^{(m-1)}\right|\le\varepsilon$.  
        Kết quả: $\lambda_k=\lambda^{(m)},\;v_k=v^{(m)}$.
      - **Nếu Case 2**, tính dãy lũy thừa chẵn:  
        $
          y_{\rm even}^{(m)} = A^2\,y_{\rm even}^{(m-1)},\quad
          y_{\rm odd}^{(m)} = A\,y_{\rm even}^{(m-1)},
        $
        rồi ước lượng
        $
          \lambda^2 \approx \frac{\left(y_{\rm even}^{(m+1)}\right)_j}
                               {\left(y_{\rm even}^{(m)}\right)_j},
        $
        chọn chỉ số $j$ sao cho $\left|y_{\rm even}^{(m)}\right|_j$ lớn nhất.  
        Lấy $\lambda_1=+\sqrt{\lambda^2},\;\lambda_2=-\lambda_1$.  
        Vector riêng:  
        $
          v_1 \propto y_{\rm even}^{(m)} + \lambda_1\,y_{\rm odd}^{(m)},\quad
          v_2 \propto y_{\rm even}^{(m)} - \lambda_1\,y_{\rm odd}^{(m)}.
        $
        Chuẩn hóa $\|v_k\|_p=1$.

   3. **Chuẩn hóa vector riêng**  
      $
        v_k \leftarrow \frac{v_k}{\|v_k\|_p},\quad
      $ nếu cần, đổi dấu sao cho phần tử lớn nhất > 0.

   4. **Lưu kết quả**  
      Ghi $\lambda_k$ vào `eigs.append(λk)`, vector `v_k` vào `vecs.append(v_k)`.

   5. **Deflation (Xuống thang)**  
      - Chọn $s = \arg\max_i |v_{k,i}|$.  
      - Tạo vector con $u^T = \frac{1}{\lambda_k}\,M_{s,\bullet}$ (hàng thứ $s$ của $M$ chia cho $\lambda_k$).  
      - Cập nhật:  
        $
          M \leftarrow M - \lambda_k\,v_k\,u^T,
        $
        giữ nguyên kích thước $M$.  
      - Đặt lại $y$ cho lần lặp kế tiếp:  
        $
          y \leftarrow (1,0,\dots,0)^T \in \mathbb{R}^n.
        $

3. **Kết thúc**  
   - Sau $n$ vòng, ta có đủ $\{\lambda_k,v_k\}_{k=1}^n$.  
   - Sắp xếp lại nếu cần theo $|\lambda|$ giảm dần.

---

**4. Ghi chú

- **Định hướng dấu**: Để dễ so sánh, có thể bắt buộc chọn dấu sao cho phần tử có độ lớn lớn nhất dương.  
- **Ổn định số**: Mỗi lần deflation làm thay đổi ma trận, sai số có thể tích lũy. Nên chọn chuẩn hợp lý và kiểm tra độ hội tụ.  
- **Áp dụng**: Thuật toán này chỉ tìm các giá trị riêng thực. Với ma trận có giá trị phức sẽ dùng phương pháp khác.

Case 3: Hai trị riêng trội là phức liên hợp

**1. Đầu vào**
- Ma trận vuông $A\in\mathbb{R}^{n\times n}$, có đầy đủ $n$ trị riêng (thực hoặc phức) và hệ vector riêng độc lập tuyến tính.  
- Giả sử hai trị riêng có độ lớn trội là phức liên hợp $\lambda_1$ và $\lambda_2=\overline{\lambda_1}$, thoả  
  $|\lambda_1|=|\lambda_2|>|\lambda_3|\ge\cdots\ge|\lambda_n|$.  
- Vector khởi tạo $Y\in\mathbb{C}^n$ (ví dụ $Y=(1,1,\dots,1)^T$), sao cho hệ số $C_1\neq0,\;C_2\neq0$ trong tổ hợp tuyến tính.  
- Ngưỡng dừng $\varepsilon>0$ và số lần lặp tối đa $K_{\max}$.  
- Chuẩn $p\in\{1,2,\infty\}$ để chuẩn hóa vectơ riêng.

**2. Đầu ra**
- Hai trị riêng phức liên hợp $\lambda_1,\lambda_2$.  
- Hai vectơ riêng tương ứng $v_1,v_2\in\mathbb{C}^n$, thỏa $\|v_k\|_p=1$.

**3. Bước thuật toán**

1. **Khởi tạo**  
   Gán $m\leftarrow0$ và $Y_m\leftarrow Y$.

2. **Tăng mũ liên tiếp**  
   – Tính tuần tự  
   $
     Y_{m+1} = A\,Y_m,\qquad
     Y_{m+2} = A\,Y_{m+1}.
   $  
   – Chọn hai chỉ số $r\neq s$ sao cho $\left|Y_{m,r}\right|$ và $\left|Y_{m,s}\right|$ tương đối lớn.

3. **Giải hệ tính $p,q$**  
   Tại mỗi hàng $j=r,s$, ta có  
   $
     \left(Y_{m+2}\right)_{j}
     + p\,\left(Y_{m+1}_\right)_{j}
     + q\,\left(Y_{m})\right)_{j}
     = 0.
   $  
   Viết dưới dạng hệ 2×2:
   $
     \begin{pmatrix}
       (Y_{m+1})_{r} & (Y_{m})_{r}\\
       (Y_{m+1})_{s} & (Y_{m})_{s}
     \end{pmatrix}
     \begin{pmatrix}p\\q\end{pmatrix}
     = -\begin{pmatrix} (Y_{m+2})_{r}\\ (Y_{m+2})_{s} \end{pmatrix}.
   $  
   Giải để được $p$ và $q$.

4. **Kiểm tra hội tụ**  
   Nếu $\lvert p - p_{\rm prev}\rvert \le \varepsilon$ và $\lvert q - q_{\rm prev}\rvert \le \varepsilon$, thì dừng, ngược lại gán  
   $p_{\rm prev}\leftarrow p,\;q_{\rm prev}\leftarrow q,\;m\leftarrow m+1$ và quay về bước 2.

5. **Tính trị riêng**  
   Sau khi hội tụ, giải phương trình bậc hai  
   $
     z^2 + p\,z + q = 0
     \;\Longrightarrow\;
     \lambda_{1,2} = \frac{-p \pm \sqrt{p^2 - 4q}}{2}.
   $

6. **Tính vector riêng**  
   – Công thức:
   $
     v_1 \;\propto\; Y_{m+1} \;-\;\lambda_1\,Y_{m}, 
     \quad
     v_2 \;\propto\; Y_{m+1} \;-\;\lambda_2\,Y_{m}.
   $  
   – Chuẩn hóa theo chuẩn $p$:  
   $
     v_k \;\leftarrow\;\frac{v_k}{\|v_k\|_p},\quad k=1,2.
   $

7. **Kết thúc**  
   Trả về hai cặp $\left(\lambda_1,v_1\right)$ và $\left(\lambda_2,v_2\right)$.

---

**Ghi chú:**  
- Chọn khởi tạo $Y$ sao cho tránh quan hệ tuyến tính đặc biệt làm nghịch hệ 2×2.  
- Hai vectơ riêng chỉ xác định đến hằng số phức; có thể áp quy ước dấu hoặc pha cho thống nhất.  
- Phương pháp chỉ áp dụng khi hai trị riêng trội là phức liên hợp.  


## Code

In [381]:
def deflate_once(A: np.ndarray, lam: float, v: np.ndarray, x0_left: np.ndarray, tol: float, max_iter: int, norm_ord: Union[int, float], precision: int) -> np.ndarray:
    """
    Deflate matrix A by removing the effect of eigenpair (lam, v) for non-symmetric A:
    1. Compute left eigenvector w of A: solve A^T w = lam w via power_method
    2. Normalize so that w^T v = 1
    3. A_new = A - lam * np.outer(v, w)
    """
    # Step 1: compute left eigenvector of A
    # Use power_method on A.T to get dominant eigenvector w
    _, w = power_method(
        A.T, x0_left, tol=tol, max_iter=max_iter,
        norm_ord=norm_ord, precision=precision
    )
    # Step 2: normalize w so that w^T v = 1
    scale = w.dot(v)
    if abs(scale) < 1e-12:
        raise ValueError("Left and right eigenvectors nearly orthogonal; cannot deflate.")
    w = w / scale
    # Step 3: rank-1 deflation
    return A - lam * np.outer(v, w)

### TH1. Trị riêng trội thực, bội đơn hoặc bội đa

In [382]:
def power_method(A,
                 x0,
                 tol=1e-6,
                 max_iter=1000,
                 norm_ord=2,
                 precision=12,
                 display=True):
    """
    Power Method for finding the dominant eigenvalue and eigenvector of A.
    Displays each iteration in a pandas DataFrame using the specified norm.

    Parameters:
      A (ndarray): Square matrix (n×n).
      x0 (ndarray): Initial guess vector (n,).
      tol (float): Convergence tolerance for eigenvalue change.
      max_iter (int): Maximum number of iterations.
      norm_ord (int, float, or str): Norm order for both eigenvalue estimation and vector normalization (e.g. 1, 2, np.inf).
      precision (int): Number of decimal places to display.

    Returns:
      eigenvalue (float): Approximate dominant eigenvalue.
      eigenvector (ndarray): Approximate eigenvector corresponding to the dominant eigenvalue.
    """
    # Normalize initial guess
    x = x0 / np.linalg.norm(x0, ord=norm_ord)

    # Step 0: record the very first y,x,λ
    y0 = x0           # or if you want y0=x0 then skip this dot
    lambda0 = np.linalg.norm(y0, ord=norm_ord)
    x0_norm = y0 / lambda0

    # history entries: [iteration, y1…yn, lambda, x1…xn]
    history = [
      [0] + y0.tolist() + [lambda0] + x0_norm.tolist()
    ]

    eigenvalue = lambda0
    n = A.shape[0]
    

    for k in range(1, max_iter+1):
        # Compute A x_{k-1}
        y = A.dot(x)
        # Estimate eigenvalue via the chosen norm
        lambda_new = np.linalg.norm(y, ord=norm_ord)
        if lambda_new == 0:
            raise ZeroDivisionError("Norm of A*x is zero; unable to continue.")
        # Normalize to get next eigenvector estimate
        x_new = y / lambda_new

        # Record iteration data: [iter, y1..yn, lambda, x1..xn]
        row = [k] + y.tolist() + [lambda_new] + x_new.tolist()
        history.append(row)

        # Check convergence on eigenvalue
        if abs(lambda_new - eigenvalue) < tol:
            break

        x = x_new
        eigenvalue = lambda_new

    # Build and display DataFrame
    cols = ['Iteration'] + [f'y{i+1}' for i in range(n)] + ['lambda'] + [f'x{i+1}' for i in range(n)]
    df = pd.DataFrame(history, columns=cols)
    if (display==True):
      print(df.to_string(
        index=False,
        float_format=lambda v: f"{v:.{precision}f}"
    ))

    return eigenvalue, x_new


### TH2. 2 trị riêng trội thực, có trị tuyệt đối bằng nhau

In [383]:
def power_method_case2(
    A,
    x0,
    tol=1e-6,
    max_iter=1000,
    norm_ord=np.inf,
    precision=12,
    display=True
):
    """
    Power Method for when the two largest eigenvalues satisfy |λ1| = |λ2| > |λ3| and λ1 = -λ2.
    Uses even powers of A to separate eigencomponents, printing y and x in one table.

    Parameters:
      A (ndarray): Square matrix (n×n).
      x0 (ndarray): Initial guess vector (n,).
      tol (float): Convergence tolerance for λ^2 change.
      max_iter (int): Maximum number of iterations for k.
      norm_ord (int, float, or str): Norm order for vector normalization.
      precision (int): Decimal places for display.

    Returns:
      lambda1 (float): Approximate dominant eigenvalue (positive).
      x1 (ndarray): Eigenvector corresponding to λ1.
      x2 (ndarray): Eigenvector corresponding to λ2 = -λ1.
    """
    n = A.shape[0]

    # Step 0: input vector and its norm^2
    y0 = x0.astype(float)
    lambda0 = np.linalg.norm(y0, ord=norm_ord)
    lambda2_old = lambda0**2
    x0_norm = y0 / lambda0

    history = []
    # record iteration 0
    history.append(
      [0] + y0.tolist() + [lambda2_old] + x0_norm.tolist() + x0_norm.tolist()
    )

    # now do the usual even‐odd power iterations
    y_even = x0.astype(float)


    for k in range(1, max_iter + 1):
        if k > 1:
            y_even = A2.dot(y_even)
        # Compute next even power
        y_even2 = A2.dot(y_even)
        # Index of largest magnitude in y_even
        j = np.argmax(np.abs(y_even))
        # Estimate λ^2
        lambda2_new = y_even2[j] / y_even[j]
        # Compute λ
        lambda1 = np.sqrt(lambda2_new)
        # Compute y_{2k+1}
        y_odd = A.dot(y_even)
        # Form eigenvectors
        x1 = y_even + lambda1 * y_odd
        x2 = y_even - lambda1 * y_odd
        # Normalize
        x1 = x1 / np.linalg.norm(x1, ord=norm_ord)
        x2 = x2 / np.linalg.norm(x2, ord=norm_ord)

        # Record iteration: [k, y_even1..n, lambda2, x1_1..n, x2_1..n]
        row = [k] + y_even.tolist() + [lambda2_new] + x1.tolist() + x2.tolist()
        history.append(row)

        # Convergence check on λ^2
        if k > 1 and abs(lambda2_new - eigen2) < tol:
            break
        eigen2 = lambda2_new

    # Display results in pandas
    cols = ['Iteration'] + [f'y{i+1}' for i in range(n)] + ['lambda^2'] + [f'x1_{i+1}' for i in range(n)] + [f'x2_{i+1}' for i in range(n)]
    df = pd.DataFrame(history, columns=cols)
    if (display==True):
      print(df.to_string(
        index=False,
        float_format=lambda v: f"{v:.{precision}f}"
    ))

    # Final eigenvalue
    lambda1_final = np.sqrt(eigen2)
    return lambda1_final, x1, x2


### Xuống thang + Lũy thừa

In [384]:
def compute_all_eigenpairs(
    A: np.ndarray,
    x0: np.ndarray,
    tol: float = 1e-6,
    max_iter: int = 200,
    norm_ord: Union[int, float] = 2,
    precision: int = 8
) -> Tuple[List[float], List[np.ndarray]]:
    """
    Compute all real eigenvalues and eigenvectors of A via deflation.

    Returns:
      eigs: list of eigenvalues (sorted by absolute value descending)
      vecs: list of corresponding eigenvectors
    """
    n = A.shape[0]
    M = A.copy().astype(float)
    y = x0.astype(float).flatten()
    eigs = []
    vecs = []

    for k in range(n):
        # Determine which power-method to apply: always use case1 for real eigenvalues
        print(f"Finding eigenpair {k+1}: matrix size {M.shape}")
        # Call power_method (case1) for new matrix M
        lam, v = power_method(
            M, y, tol=tol, max_iter=max_iter,
            norm_ord=norm_ord, precision=precision, display=False
        )
        eigs.append(lam)
        vecs.append(v)
        # Deflate: pass y and solver parameters to compute left eigenvector
        M = deflate_once(M, lam, v, y, tol, max_iter, norm_ord, precision)
        # Reset y to a standard initial guess in the deflated space
        y = x0.astype(float).flatten()

    # Sort eigenpairs by descending magnitude of eigenvalue
    pairs = sorted(zip(eigs, vecs), key=lambda ev: -abs(ev[0]))
    eigs_sorted, vecs_sorted = zip(*pairs)

 # Display final results: eigenvalue and eigenvector on same line
    print("All eigenvalues and eigenvectors (sorted):")
    for i, (lam, vec) in enumerate(zip(eigs_sorted, vecs_sorted), start=1):
        vec_str = ", ".join(f"{x:.{precision}f}" for x in vec)
        print(f"  λ{i} = {lam:.{precision}f}, v = [{vec_str}]")

    return list(eigs_sorted), list(vecs_sorted)

### TH3. 2 trị riêng phức có module bằng nhau

In [385]:
def power_method_case3(
    A: np.ndarray,
    x0: np.ndarray,
    tol: float = 1e-6,
    max_iter: int = 100,
    norm_ord: Union[int, float, str] = np.inf,
    precision: int = 12
) -> (complex, complex, np.ndarray, np.ndarray):
    """
    Power‐Method variant for the case where the two largest eigenvalues of A
    are complex conjugates (|λ1| = |λ2| > |λ3| ≥ ...),
    i.e. λ2 = conj(λ1).  We estimate p and q in the quadratic Z^2 + pZ + q = 0
    and then recover λ1, λ2 and their eigenvectors.

    Parameters:
    - A (ndarray): square matrix (n×n), real or complex.
    - x0 (ndarray): initial guess vector (n,), complex or real.
    - tol (float): tolerance on p and q convergence.
    - max_iter (int): maximum number of successive powers k to attempt.
    - norm_ord: norm order for eigenvector normalization (1, 2, np.inf, etc.).
    - precision (int): number of decimal places to print in pandas output.

    Returns:
    - lambda1, lambda2 (complex): the two conjugate eigenvalues of largest magnitude.
    - v1, v2 (ndarray): normalized eigenvectors corresponding to λ1 and λ2 (n,).
    """

    n = A.shape[0]
    # initial y vectors
    y_m = x0.astype(complex).reshape(n)
    y_m1 = A.dot(y_m)
    y_m2 = A.dot(y_m1)

    history = []
    # Step 0: solve for initial p0,q0 from rows 0,1
    M0 = np.array([[y_m1[0], y_m[0]], [y_m1[1], y_m[1]]], dtype=complex)
    b0 = -np.array([y_m2[0], y_m2[1]], dtype=complex)
    p0, q0 = np.linalg.solve(M0, b0)
    history.append([0] + y_m.tolist() + y_m1.tolist() + y_m2.tolist() + [p0, q0])

    p_old, q_old = p0, q0

    for m in range(max_iter):
        # At this point: y_m = A^m y, y_m1 = A^(m+1) y, y_m2 = A^(m+2) y

        # Pick two distinct indices (e.g., 0 and 1) to form 2×2 system:
        i1, i2 = 0, 1
        # Solve for p, q in: y2 + p * y1 + q * y = 0  at rows i1 and i2
        M = np.array([[y_m1[i1], y_m[i1]],
                      [y_m1[i2], y_m[i2]]], dtype=complex)
        b = -np.array([y_m2[i1], y_m2[i2]], dtype=complex)

        # Solve 2×2 linear system
        try:
            p, q = np.linalg.solve(M, b)
        except np.linalg.LinAlgError:
            raise RuntimeError(
                f"Singular 2×2 system encountered at iteration m={m}; "
                "choose a different pair of pivot indices or check A."
            )

        # Record iteration data: [m, y_m, y_m1, y_m2, p, q]
        row = [m] + y_m.tolist() + y_m1.tolist() + y_m2.tolist() + [p, q]
        history.append(row)

        # Check convergence on p and q
        if m > 0 and (abs(p - p_old) < tol and abs(q - q_old) < tol):
            break

        p_old, q_old = p, q

        # Move to next power: shift (m→m+1)
        y_m = y_m1
        y_m1 = y_m2
        y_m2 = A.dot(y_m1)

    # Build DataFrame columns
    cols = ['m'] + [f'y_m{i+1}' for i in range(n)] \
           + [f'y_m1_{i+1}' for i in range(n)] \
           + [f'y_m2_{i+1}' for i in range(n)] \
           + ['p', 'q']
    df = pd.DataFrame(history, columns=cols)

    # Print iteration history with specified precision
    print(df.to_string(index=False,
                       float_format=lambda v: f"{v:.{precision}f}"))

    # Now solve the quadratic for eigenvalues
    # z^2 + p z + q = 0  =>  z = [-p ± sqrt(p^2 - 4q)]/2
    discriminant = p * p - 4.0 * q
    sqrt_disc = np.sqrt(discriminant)
    lambda1 = (-p + sqrt_disc) / 2.0
    lambda2 = (-p - sqrt_disc) / 2.0

    # Build eigenvectors from formula:
    #   v1 ∝ A^(m+1) y  - λ1 * A^m y
    #   v2 ∝ A^(m+1) y  - λ2 * A^m y
    v1 = y_m1 - lambda1 * y_m
    v2 = y_m1 - lambda2 * y_m

    # Normalize each eigenvector
    v1 = v1 / np.linalg.norm(v1, ord=norm_ord)
    v2 = v2 / np.linalg.norm(v2, ord=norm_ord)

    # Display eigenvectors in a small DataFrame
    df_eigs = pd.DataFrame({
        'v1': v1,
        'v2': v2
    })

    return lambda1, lambda2, df_eigs


## Kết quả

In [386]:
# Define your square matrix A
A = input_matrix('PWDF_input_A.txt', convert_fractions=False)
print("\nMatrix A:"); output_matrix(A)


Matrix A:
 6.7562  4.4584  5.1176  3.5945  3.4311
 4.4584  4.3051  4.3327  1.9363  1.5788
 5.1176  4.3327  5.1287  3.5097  3.3354
 3.5945  1.9363  3.5097  4.8411  4.8972
 3.4311  1.5788  3.3354  4.8972  5.0688


### TH1+TH2

In [387]:
# Define an initial guess vector x0
A = input_matrix('PWDF_input_A.txt', convert_fractions=False)
x0 = np.array([1, 1, 1, 1, 1])

# Call the power_method function
eigs, vecs = compute_all_eigenpairs(A, x0)

# Display final results

Finding eigenpair 1: matrix size (5, 5)
 Iteration          y1         y2         y3         y4         y5      lambda         x1         x2         x3         x4         x5
         0  1.00000000 1.00000000 1.00000000 1.00000000 1.00000000  2.23606798 0.44721360 0.44721360 0.44721360 0.44721360 0.44721360
         1 10.44592572 7.42879920 9.58114879 8.39813467 8.18906231 19.84167940 0.52646379 0.37440375 0.48287993 0.42325725 0.41272022
         2 10.63480521 7.52236133 9.65504953 8.38229991 8.17282794 19.99907627 0.53176482 0.37613544 0.48277477 0.41913435 0.40866027
         3 10.64905260 7.53860191 9.66112996 8.36449673 8.15262983 20.00001127 0.53245233 0.37692988 0.48305623 0.41822460 0.40763126
         4 10.65187911 7.54292055 9.66290881 8.36005060 8.14751068 20.00005905 0.53259238 0.37714491 0.48314401 0.41800130 0.40737433
         5 10.65254908 7.54401302 9.66336675 8.35893921 8.14622762 20.00006201 0.53262580 0.37719948 0.48316684 0.41794566 0.40731012
         6 10.65271467

### TH3

In [388]:
A3 = input_matrix('PWDF_input_A3.txt', convert_fractions=False)
x0 = np.array([-1, 1, 0, 0], dtype=float)

lambda1, lambda2, df_eigs = power_method_case3(
    A3,
    x0,
    tol=1e-6,
    max_iter=50,
    norm_ord=2,
    precision=8
)
print("Estimated eigenvalue:")
print("λ1 ≈", lambda1)
print("λ2 ≈", lambda2)
print("Estimated eigenvectors (normalized):")
print(df_eigs.to_string(float_format=lambda v: f"{v:.{precision}f}"))

 m                                   y_m1                                   y_m2                                  y_m3                            y_m4                                 y_m1_1                                  y_m1_2                                  y_m1_3                          y_m1_4                                  y_m2_1                                  y_m2_2                                  y_m2_3                           y_m2_4                      p                       q
 0                -1.00000000+0.00000000j                 1.00000000+0.00000000j                0.00000000+0.00000000j          0.00000000+0.00000000j                 3.00000000+0.00000000j                  2.00000000+0.00000000j                 -1.00000000+0.00000000j          1.00000000+0.00000000j                 -4.00000000+0.00000000j                -30.00000000+0.00000000j                 -1.00000000+0.00000000j          -2.00000000+0.00000000j 6.80000000-0.00000000j 16.40000000-0.000000